In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
df = sns.load_dataset('tips')

In [3]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [5]:
df.isnull().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

In [6]:
df['time'].value_counts()

time
Dinner    176
Lunch      68
Name: count, dtype: int64

In [7]:
df['day'].value_counts()

day
Sat     87
Sun     76
Thur    62
Fri     19
Name: count, dtype: int64

In [8]:
df.sex.value_counts()

sex
Male      157
Female     87
Name: count, dtype: int64

In [9]:
df.smoker.value_counts()

smoker
No     151
Yes     93
Name: count, dtype: int64

In [10]:
# depedent and indepedent feature
df.columns

Index(['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size'], dtype='object')

In [11]:
X = df[['tip', 'sex', 'smoker', 'day', 'time', 'size']]
y = df['total_bill']

In [12]:
X.head()

,tip,sex,smoker,day,time,size
0,1.01,Female,No,Sun,Dinner,2
1,1.66,Male,No,Sun,Dinner,3
2,3.50,Male,No,Sun,Dinner,3
3,3.31,Male,No,Sun,Dinner,2
4,3.61,Female,No,Sun,Dinner,4


In [13]:
# first do train test split for data leakage
from sklearn.model_selection import train_test_split

In [14]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.20,random_state=42)

In [15]:
X_train.shape,X_test.shape

((195, 6), (49, 6))

In [16]:
# lets do lable encoding and one hot encoding 
from sklearn.preprocessing import LabelEncoder,OneHotEncoder 

In [17]:
lab1 = LabelEncoder()
lab2 = LabelEncoder()
lab3 = LabelEncoder()

In [18]:
X_train['sex'] = lab1.fit_transform(X_train['sex'])
X_test['sex'] = lab1.transform(X_test['sex'])

In [19]:
X_train['time'] = lab2.fit_transform(X_train['time'])
X_test['time'] = lab2.transform(X_test['time'])

In [20]:
X_train['smoker'] = lab3.fit_transform(X_train['smoker'])
X_test['smoker'] = lab3.transform(X_test['smoker'])

In [21]:
X_train.head()

,tip,sex,smoker,day,time,size
228,2.72,1,0,Sat,0,2
208,2.03,1,1,Sat,0,2
96,4.00,1,1,Fri,0,2
167,4.50,1,0,Sun,0,4
84,2.03,1,0,Thur,1,2


In [22]:
# lets do one hot encoding along with columnTransfomer
from sklearn.compose import ColumnTransformer


In [23]:
ct = ColumnTransformer(transformers=[('onehot',OneHotEncoder(drop='first'),[3])],remainder='passthrough')

In [24]:
X_train_trans = ct.fit_transform(X_train)

In [25]:
X_train_trans

array([[1., 0., 0., ..., 0., 0., 2.],
       [1., 0., 0., ..., 1., 0., 2.],
       [0., 0., 0., ..., 1., 0., 2.],
       ...,
       [0., 0., 0., ..., 1., 0., 2.],
       [0., 1., 0., ..., 1., 0., 2.],
       [1., 0., 0., ..., 1., 0., 3.]])

In [26]:
X_test_trans = ct.transform(X_test)

In [27]:
type(X_train_trans)

numpy.ndarray

In [28]:
# model tarining
from sklearn.svm import SVR


In [29]:
ovr = SVR(verbose=1)

In [30]:
ovr.fit(X_train_trans,y_train)

[LibSVM]

SVR(verbose=1)

In [31]:
ovr.score(X_test_trans,y_test)

0.5502250005312896

In [49]:
y_pred = ovr.predict(X_test_trans)
from sklearn.metrics import r2_score
score = r2_score(y_test,y_pred)
score

0.5502250005312896

## hyper parameter tunign

In [32]:
from sklearn.model_selection import GridSearchCV

In [33]:
param_grid = {
    'kernel': ['linear','rbf','poly'],
    'C' : [0.1,1.0,10,100],
    'degree' : [2,3,4]

}

In [34]:
gird = GridSearchCV(SVR(),param_grid=param_grid,scoring='accuracy',verbose=3)

In [42]:
len(X_train_trans),len(y_train),len(X_test_trans),len(y_test)

(195, 195, 49, 49)

In [50]:
import warnings
warnings.filterwarnings('ignore')

In [51]:
gird.fit(X_train_trans,y_train)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
[CV 1/5] END ......C=0.1, degree=2, kernel=linear;, score=nan total time=   0.0s
[CV 2/5] END ......C=0.1, degree=2, kernel=linear;, score=nan total time=   0.0s
[CV 3/5] END ......C=0.1, degree=2, kernel=linear;, score=nan total time=   0.0s
[CV 4/5] END ......C=0.1, degree=2, kernel=linear;, score=nan total time=   0.0s
[CV 5/5] END ......C=0.1, degree=2, kernel=linear;, score=nan total time=   0.0s
[CV 1/5] END .........C=0.1, degree=2, kernel=rbf;, score=nan total time=   0.0s
[CV 2/5] END .........C=0.1, degree=2, kernel=rbf;, score=nan total time=   0.0s
[CV 3/5] END .........C=0.1, degree=2, kernel=rbf;, score=nan total time=   0.0s
[CV 4/5] END .........C=0.1, degree=2, kernel=rbf;, score=nan total time=   0.0s
[CV 5/5] END .........C=0.1, degree=2, kernel=rbf;, score=nan total time=   0.0s
[CV 1/5] END ........C=0.1, degree=2, kernel=poly;, score=nan total time=   0.0s
[CV 2/5] END ........C=0.1, degree=2, kernel=po

GridSearchCV(estimator=SVR(),
             param_grid={'C': [0.1, 1.0, 10, 100], 'degree': [2, 3, 4],
                         'kernel': ['linear', 'rbf', 'poly']},
             scoring='accuracy', verbose=3)

In [52]:
gird.best_params_

{'C': 0.1, 'degree': 2, 'kernel': 'linear'}

In [53]:
y_pred = gird.predict(X_test_trans)

In [54]:
from sklearn.metrics import r2_score
score = r2_score(y_test,y_pred)

In [55]:
score

0.6149102116295675